In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import geopandas as gpd
from pandas import DataFrame
from shapely.geometry import Point, Polygon

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
#phデータをロード
df_ph = pd.read_excel('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/土壌図データ（ベクトルデータ）/全国版_土壌/全国版/代表断面/代表土壌断面データ（Excel）/03岩手.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [5]:
#カラムを厳選
df_ph = df_ph[['KEY01', 'KEY02', 'PHH2O']]

In [6]:
#カラム名を指定の名前に変換
df_ph.rename(columns = {'PHH2O':'soil_ph'}, inplace=True)

In [7]:
#結合するときにint型とobject型であっていなかったので修正
df_ph['KEY01'] = df_ph['KEY01'].astype(str)

In [8]:
#phのgeometryをロード
gdf_point = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/土壌図データ（ベクトルデータ）/全国版_土壌/全国版/代表断面/代表土壌地点図（SHP）/Point全国.shp', encoding='shift-jis')

In [9]:
#カラムを厳選
gdf_point = gdf_point[['地点コード', 'geometry']]

In [10]:
#結合するためにカラム名を変更
gdf_point.rename(columns = {'地点コード':'KEY01'}, inplace=True)

In [11]:
#結合
df_ph = pd.merge(df_ph, gdf_point, on='KEY01', how = 'left')

In [12]:
#dfをgdfに変換
gdf_ph = gpd.GeoDataFrame(df_ph, geometry='geometry')

In [13]:
#座標変換
if gdf_ph.crs is None:
	gdf_ph.set_crs(epsg=4326, inplace=True)

In [14]:
#土壌図データにgeometryを追加したshpファイルをアップロード
gdf_ph.to_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市/奥州市　土壌/土壌ph/土壌図データ.shp')

In [15]:
gdf_soil_type = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/地形・表層地質データ/岩手/030土壌分類（ポリゴン）.shp')

In [16]:
#座標返還
if gdf_soil_type.crs is None:
	gdf_soil_type.set_crs(epsg=4326, inplace=True)

In [17]:
#カラム名を指定の名前に変換
gdf_soil_type.rename(columns={'属性1':'土壌タイプ1', '属性2':'土壌タイプ2'}, inplace=True)

In [18]:
#連番をつける
number = [i for i in range(len(gdf_soil_type))]
gdf_soil_type['num'] = number

In [19]:
#土壌タイプと土壌phを結合
gdf_soil_tmp = gpd.sjoin(gdf_soil_type, gdf_ph, op='contains',  how='left')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [22]:
#phの平均を計算

#ph以外の計算しないカラムのリストを作成
other_columns = gdf_soil_tmp.columns.to_list()
other_columns.remove('soil_ph')

#処理する内容の辞書を作成
agg_dict = {'soil_ph': 'mean'}

agg_dict.update({col: 'first' for col in other_columns})

# 'num'でグループ化して'PHH2O'の平均を計算し、'other_column'も保持
gdf_soil = gdf_soil_tmp.groupby('num', as_index=False).agg(agg_dict)

In [23]:
gdf_soil[gdf_soil.index.duplicated()].index.value_counts()

Series([], dtype: int64)

In [24]:
gdf_soil[~gdf_soil['soil_ph'].isna()]

,soil_ph,OBJECTID,土壌タイプ1,土壌タイプ2,geometry,num,index_right,KEY01,KEY02
352,7.700000,374.0,グライ土,グライ土壌,"POLYGON ((141.70745 38.96411, 141.70816 38.963...",352,97.0,100321009001,1.003210e+13
401,6.000000,427.0,褐色森林土,乾性褐色森林土壌（赤褐系）,"POLYGON ((141.70112 38.97743, 141.70015 38.976...",401,123.0,100321009052,1.003210e+13
449,5.450000,475.0,褐色森林土,褐色森林土壌,"POLYGON ((141.70436 38.96418, 141.70284 38.962...",449,98.0,100321009002,1.003210e+13
493,5.750000,519.0,黒ボク土,淡色黒ボク土壌,"POLYGON ((141.61555 39.00069, 141.61647 39.000...",493,125.0,100321009053,1.003210e+13
595,5.933333,629.0,灰色低地土,灰色低地土壌,"POLYGON ((141.65328 39.02091, 141.65681 39.019...",595,100.0,100321009003,1.003210e+13
...,...,...,...,...,...,...,...,...,...
7051,6.550000,8382.0,黒ボク土,粗粒黒ボク土壌,"POLYGON ((141.43193 40.35297, 141.43187 40.352...",7051,556.0,100350113051,1.003501e+13
7053,6.148000,8384.0,黒ボク土,黒ボク土壌,"POLYGON ((141.70370 40.35908, 141.70078 40.357...",7053,71.0,100320718008,1.003207e+13
7359,6.157143,5390.0,褐色森林土,褐色森林土壌,"POLYGON ((141.57031 39.57124, 141.56916 39.570...",7359,487.0,100348626003,1.003486e+13
7440,5.700000,4410.0,岩屑土,岩屑性土壌,"POLYGON ((141.66010 39.59236, 141.66134 39.591...",7440,527.0,100348726057,1.003487e+13


In [25]:
gdf_soil.loc[7053:7053,]

,soil_ph,OBJECTID,土壌タイプ1,土壌タイプ2,geometry,num,index_right,KEY01,KEY02
7053,6.148,8384.0,黒ボク土,黒ボク土壌,"POLYGON ((141.70370 40.35908, 141.70078 40.357...",7053,71.0,100320718008,1.003207e+13


In [26]:
gdf_soil.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市/奥州市　土壌/土壌ph/土壌ph.csv', index=False)

#結合の確認

In [ ]:
gdf_soil_tmp[gdf_soil_tmp.index.duplicated()].index.value_counts()

In [ ]:
gdf_soil_tmp.loc[5658,]